In [2]:
import numpy as np
import LoadData as ld
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
import mne
from time import perf_counter
import keras
import tensorflow as tf


def createEpoch(feature_data, epoch_size, epoch_window):
    '''
    this creates epoch for CSP data which will be lateer
    used for LDA
    '''
    epochs = []
    n_samples = feature_data.shape[1]
    for i in range(0, n_samples, epoch_window):
        if n_samples - i < epoch_size:
            break

        epoch_data = feature_data[:, i: i + epoch_size]
        epochs.append(epoch_data)
    
    
    epochs_array = np.array(epochs)

    return epochs_array



def createChannelInfo(no_channels):
    '''This function creates an info object 
    that is required later to convert the eeg data from np data
    to mne.io.raw data.

    no_channels: number of channels used in the eeg data

    returns info object
    '''
    ch_names = []
    for i in range(no_channels):
        ch_names.append('Channel' +"-"+ str(i))   #for this code I have just created dummy channel names. We can have actual channel names
    info = mne.create_info(ch_names = ch_names, sfreq = 250, ch_types='eeg') 
    return info 

def bandPassFilterReshape(raw_data, info):
    '''
    First converts the np data into mne.io.raw data and
    uses createEpoch funcn to convert the data into epochs
    raw_data : eeg data
    info: info object we created before which has information about the channel
    
    returns epoch data which will be applied for CSP
    '''
    raw_data = np.transpose(raw_data)
    features = mne.io.RawArray(raw_data, info)
    features.filter(7.0, 40.0)
    features_data = features.get_data()
    # features_data = np.transpose(features_data)
    features_data = createEpoch(features_data, 250, 64)
    return features_data


In [8]:
raw_data_hand = ld.loadData()   
raw_data_feet = ld.loadData()    #loading eeg data of any two movements of ur choosing

test_data_hand = ld.loadData()
test_data_feet = ld.loadData() 

info = createChannelInfo(22)     #we will need the info object to convert the eeg data from np data to raw_eeg data which we will need for bandpass filtering

features_hand = bandPassFilterReshape(raw_data_hand, info)   #this converts the eeg data into epochs, which will be needed for CSP
features_feet = bandPassFilterReshape(raw_data_feet, info)

features_test_hand = bandPassFilterReshape(test_data_hand, info)
features_test_feet = bandPassFilterReshape(test_data_feet, info)

The file name provided is correct. Thank you!
The file name provided is correct. Thank you!
The file name provided is correct. Thank you!
The file name provided is correct. Thank you!
Creating RawArray with float64 data, n_channels=22, n_times=51000
    Range : 0 ... 50999 =      0.000 ...   203.996 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 413 samples (1.652 s)

Creating RawArray with float64 data, n_channels=22, n_times=51750
    Range : 0 ... 51749 =      0.000 ...   206.996 secs
Read

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Creating RawArray with float64 data, n_channels=22, n_times=52500
    Range : 0 ... 52499 =      0.000 ...   209.996 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 413 samples (1.652 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


In [9]:
features =  np.concatenate((features_hand, features_feet))
label = np.concatenate((np.zeros(features_hand.shape[0]),np.ones(features_feet.shape[0])))

features_test =  np.concatenate((features_test_hand, features_test_feet))
label_test = np.concatenate((np.zeros(features_test_hand.shape[0]),np.ones(features_test_feet.shape[0])))


In [10]:
csp = mne.decoding.CSP(n_components = 21, reg = None, log = True, norm_trace = False)
features_transform = csp.fit_transform(features, label)
features_test_transform = csp.transform(features_test)

dataset = tf.data.Dataset.from_tensor_slices((features_transform, label))
dataset = dataset.shuffle(buffer_size= features_transform.shape[0]).batch(32)


dataset_test = tf.data.Dataset.from_tensor_slices((features_test_transform, label_test))
dataset_test = dataset_test.shuffle(buffer_size = features_test_transform.shape[0]).batch(32)

Computing rank from data with rank=None


    Using tolerance 55 (2.2e-16 eps * 22 dim * 1.1e+16  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 53 (2.2e-16 eps * 22 dim * 1.1e+16  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.


In [11]:
input = keras.Input(shape=(features_transform.shape[1],))

hl1 =  keras.layers.Dense(128, activation= keras.layers.LeakyReLU(0.3))(input)

hl2 = keras.layers.Dense(128, activation= keras.layers.LeakyReLU(0.3))(hl1)

hl3 = keras.layers.Dense(128, activation = keras.layers.LeakyReLU(0.3)) (hl2)


output = keras.layers.Dense(2, activation = "softmax")(hl3)


model =  keras.Model(input, output)
print(model.summary())

In [12]:
model.compile(optimizer="adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])

model.fit(dataset, epochs = 50, validation_data = dataset_test)

Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7641 - loss: 0.4533 - val_accuracy: 0.9162 - val_loss: 0.2337
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8747 - loss: 0.2984 - val_accuracy: 0.9186 - val_loss: 0.2126
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8921 - loss: 0.2613 - val_accuracy: 0.8755 - val_loss: 0.3087
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8989 - loss: 0.2678 - val_accuracy: 0.8964 - val_loss: 0.2436
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9101 - loss: 0.2426 - val_accuracy: 0.9260 - val_loss: 0.1996
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9167 - loss: 0.2161 - val_accuracy: 0.8656 - val_loss: 0.3347
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9111 - loss: 0.2197 - val_accuracy: 0.9205 - val_loss: 0.2094
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9213 - loss: 0.1968 - val_accuracy: 0.9174 - val_loss: